# Overview

This notebook explores the feasability of implementing ACOLITE  in Earth Engine. ACOLITE is a atmospheric correction algorithm for aquatic applications. A Python implementation of ACOLITE is available at https://github.com/acolite.

# Notebook Setup



In [ ]:
!pip install netCDF4

# Analysis

In [ ]:
from pprint import pprint

## Look Up Tables approximation

The ACOLITE Python implementation makes extensive use of look up tables (LUTs), both generic (sensor agnostic) and sensor specific. See: https://github.com/acolite/acolite_luts

This section investigates whether the LUTs can be approximated by trained [supervised classification models](https://developers.google.com/earth-engine/guides/classification), which can be efficiently used in Earth Engine scripts.

In [ ]:
url = "https://github.com/acolite/acolite_luts/blob/main/ACOLITE-LUT-202102/S2A_MSI/ACOLITE-LUT-202102-MOD2-1100mb_S2A_MSI.nc?raw=true"

## Download file

In [ ]:
import urllib.request
import shutil

file_name = 'local.nc'

# Download the file from `url` and save it locally under `file_name`:
with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

## Inspect the netCDF

In [ ]:
# Read data from an opendap server
import netCDF4

dataset = netCDF4.Dataset(file_name, mode='r')

# List the variables
pprint(dataset.variables)

{'1': <class 'netCDF4._netCDF4.Variable'>
float32 1(par, azi, thv, ths, wnd, tau)
unlimited dimensions: 
current shape = (22, 13, 13, 16, 1, 16)
filling on, default _FillValue of 9.969209968386869e+36 used,
 '10': <class 'netCDF4._netCDF4.Variable'>
float32 10(par, azi, thv, ths, wnd, tau)
unlimited dimensions: 
current shape = (22, 13, 13, 16, 1, 16)
filling on, default _FillValue of 9.969209968386869e+36 used,
 '11': <class 'netCDF4._netCDF4.Variable'>
float32 11(par, azi, thv, ths, wnd, tau)
unlimited dimensions: 
current shape = (22, 13, 13, 16, 1, 16)
filling on, default _FillValue of 9.969209968386869e+36 used,
 '12': <class 'netCDF4._netCDF4.Variable'>
float32 12(par, azi, thv, ths, wnd, tau)
unlimited dimensions: 
current shape = (22, 13, 13, 16, 1, 16)
filling on, default _FillValue of 9.969209968386869e+36 used,
 '2': <class 'netCDF4._netCDF4.Variable'>
float32 2(par, azi, thv, ths, wnd, tau)
unlimited dimensions: 
current shape = (22, 13, 13, 16, 1, 16)
filling on, default _

In [ ]:
dataset.dimensions

{'azi': <class 'netCDF4._netCDF4.Dimension'>: name = 'azi', size = 13,
 'par': <class 'netCDF4._netCDF4.Dimension'>: name = 'par', size = 22,
 'tau': <class 'netCDF4._netCDF4.Dimension'>: name = 'tau', size = 16,
 'ths': <class 'netCDF4._netCDF4.Dimension'>: name = 'ths', size = 16,
 'thv': <class 'netCDF4._netCDF4.Dimension'>: name = 'thv', size = 13,
 'wnd': <class 'netCDF4._netCDF4.Dimension'>: name = 'wnd', size = 1}

In [ ]:
# Inspect the first variable
variable = dataset.variables['1']

print(f'variable.shape: {variable.shape}')

# print the first few values
print(variable[21][0][0][0][0][0])

variable.shape: (22, 13, 13, 16, 1, 16)
0.10015615


In [ ]:
variable.dimensions

('par', 'azi', 'thv', 'ths', 'wnd', 'tau')